Votes are rounded above 999 => need to load the review page to get exact values

Setting language to English not necessary from the DataLab?

Check if the number of scrapped matches the number of reviews displayed on the main page

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from src.utils.db import PostgreSQLDatabase

import pandas as pd
import re
import time
import tqdm

In [2]:
movie_id = '6208148'

In [3]:
# Initialize Selenium WebDriver in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")
driver = webdriver.Chrome(options=chrome_options)
print(f"[INFO] Launching browser")

[INFO] Launching browser


In [4]:
# Load main page
driver.get(f"https://www.imdb.com/title/tt{movie_id}")
print(f"[INFO] Loading IMDb main page for movie #{movie_id}")
            
            # Wait for and extract title
title_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//span[@data-testid="hero__primary-text"]'))
)
movie_title = title_element.text.strip()
print(f"[INFO] Extracting movie title: {movie_title}")
            
            # Extract release date
release_date_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//li[@data-testid="title-details-releasedate"]//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]'))
)
release_date = release_date_element.text.split(" (")[0].strip()
print(f"[INFO] Extracting release date: {release_date}")

[INFO] Loading IMDb main page for movie #6208148
[INFO] Extracting movie title: Snow White
[INFO] Extracting release date: March 21, 2025


In [5]:
# Load review page
driver.get(f"https://www.imdb.com/title/tt{movie_id}/reviews")
time.sleep(3)  # Allow page to load
print(f"[INFO] Loading IMDb reviews page for movie #{movie_id}")

# Extract the number of reviews
reviews_element = driver.find_element(By.XPATH, '//div[@data-testid="tturv-total-reviews"]')
if reviews_element:
    total_reviews = reviews_element.text.strip().split(" reviews")[0]  # Remove the text
    total_reviews = int(total_reviews.replace(",", ""))  # Remove the comma for numbers >999
else:
    total_reviews = None

print(f"[INFO] Found {total_reviews} reviews")

[INFO] Loading IMDb reviews page for movie #6208148
[INFO] Found 1386 reviews


In [6]:
# Load review page
driver.get(f"https://www.imdb.com/title/tt{movie_id}/reviews")
time.sleep(3)  # Allow page to load
print(f"[INFO] Loading IMDb reviews page for movie #{movie_id}")

[INFO] Loading IMDb reviews page for movie #6208148


In [7]:
# Click the button to display all reviews
if total_reviews > 25:
    try:
        # Wait for the button that specifically contains "All"
        all_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//span[contains(@class, "ipc-see-more")]//button[.//span[contains(text(), "All")]]'))
        )
    
        # Click the button using JavaScript to avoid interception issues
        driver.execute_script("arguments[0].click();", all_button)
        print(f"[INFO] Clicking the button to display all reviews")

    except Exception as e:
        print(f"[WARNING] Button for displaying the reviews not found or not clickable: {e}")

[INFO] Clicking the button to display all reviews


In [8]:
# Click all the spoiler buttons until all reviews are displayed entirely
spoiler_buttons = driver.find_elements(By.CLASS_NAME, "review-spoiler-button")
print(f"[INFO] Found {len(spoiler_buttons)} spoiler buttons to click")
    
for i, spoiler_button in enumerate(tqdm.tqdm(spoiler_buttons, desc="Clicking Spoilers", unit="button")):
    try:
        ActionChains(driver).move_to_element(spoiler_button).click().perform()
        time.sleep(1)  # Add a small delay to ensure clicks register properly
    except Exception as e:
        print(f"[ERROR] Could not click spoiler button {i+1}: {e}")
        continue

[INFO] Found 8 spoiler buttons to click


Clicking Spoilers: 100%|██████████| 8/8 [00:14<00:00,  1.87s/button]


In [9]:
# Extract reviews
soup = BeautifulSoup(driver.page_source, "html.parser")
data = []

# Find all review articles
reviews = soup.find_all("article", class_="user-review-item")

# Loop through each review and extract information
for review in reviews:
    # 1. Extract the review identifier (integer between /rw and /?)
    permalink_tag = review.find("a", class_="ipc-link ipc-link--base", attrs={"data-testid": "permalink-link"})
    if permalink_tag:
        identifier_match = re.search(r"/rw(\d+)", permalink_tag["href"])
        if identifier_match:
            review_id = identifier_match.group(1)
        else:
            review_id = None
    else:
        review_id = None

    # 2. Extract the review date (from <li> tag with class 'review-date')
    date_tag = review.find("li", class_="ipc-inline-list__item review-date")
    review_date = date_tag.get_text(strip=True) if date_tag else None

    # 3. Extract the review author (from <a> tag with class 'author-link')
    author_tag = review.find("a", class_="ipc-link ipc-link--base", attrs={"data-testid": "author-link"})
    author_name = author_tag.get_text(strip=True) if author_tag else None

    # 4. Extract the upvotes and downvotes (from ipc-voting__label__count classes)
    upvotes_tag = review.find("span", class_="ipc-voting__label__count--up")
    downvotes_tag = review.find("span", class_="ipc-voting__label__count--down")
    upvotes = upvotes_tag.get_text(strip=True) if upvotes_tag else 0
    downvotes = downvotes_tag.get_text(strip=True) if downvotes_tag else 0

    # 5. Extract the review text
    spoiler_content_tag = review.find("div", {"data-testid": "review-spoiler-content"})
    
    if spoiler_content_tag:
        # If the spoiler content exists, extract the inner HTML of the review
        review_text = spoiler_content_tag.find("div", class_="ipc-html-content-inner-div")
        review_text = review_text.get_text(separator="\n", strip=True) if review_text else None
    else:
        # If no spoiler content, extract the regular review text
        review_text_tag = review.find("div", class_="ipc-overflowText--children")
        review_text = review_text_tag.get_text(separator="\n", strip=True) if review_text_tag else None

    # 6. Extract the review title (from <h3> inside a <div> with class 'ipc-title')
    title_tag = review.find("div", class_="ipc-title").find("h3", class_="ipc-title__text")
    review_title = title_tag.get_text(strip=True) if title_tag else None

    # 7. Extract the rating (from <span> with class 'ipc-rating-star--maxRating')
    rating_tag = review.find("span", class_="ipc-rating-star--maxRating")
    rating = rating_tag.previous_sibling.get_text(strip=True) if rating_tag else None

    # 8. Append data to the list
    data.append({
        "movie_id": movie_id,
        "review_id": review_id,
        "author": author_name, 
        "title": review_title,
        "text": review_text,
        "rating": rating,
        "date": review_date,
        "upvotes": upvotes, 
        "downvotes": downvotes, 
        "scrapping_timestamp": datetime.now().strftime("%Y%m%d_%H%M%S")
    })

# Create a dataframe from the collected data
reviews_df = pd.DataFrame(data)

In [10]:
len(reviews_df)

550

In [ ]:
# select rows with upvotes or downvotes ending with K
# generate their adress
# scrap vote counts
# https://www.imdb.com/review/rw10392322/?ref_=tturv_perm_1

In [ ]:
review_id = '10392322'
driver.get(f"https://www.imdb.com/review/rw{review_id}/")
time.sleep(3)  # Allow page to load
print(f"[INFO] Loading IMDb page for review #{review_id}")

In [ ]:
votes_element = driver.find_element(By.XPATH, '//div[@class="actions text-muted"]')
votes_text = votes_element.text.strip()
votes_match = re.search(r'([\d,]+) out of ([\d,]+)', votes_text)

if votes_match:
    upvotes = int(votes_match.group(1).replace(',', ''))
    all_votes = int(votes_match.group(2).replace(',', ''))
    downvotes = all_votes - upvotes
    print(f"Helpful votes: {votes_helpful} out of {votes_total} and {downvotes}")

In [ ]:
driver.quit()

In [ ]:
def convert_to_int(value):
    if value is None:
        return 0
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)
    try:
        return int(value)
    except ValueError:
        return 0

reviews_df['upvotes'] = reviews_df['upvotes'].apply(convert_to_int)
reviews_df['downvotes'] = reviews_df['downvotes'].apply(convert_to_int)

In [ ]:
reviews_list = reviews_df.apply(lambda row: (
    row['movie_id'], row['review_id'], 
    f"{row['author']}", f"{row['title']}", 
    f'''{row['text']}''', row['rating'], 
    f"{row['date']}", row['upvotes'], row['downvotes'], 
    f"{row['scrapping_timestamp']}"
), axis=1).tolist()

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

db = PostgreSQLDatabase()
db.connect()
db.insert_data('movies', [(movie_id, movie_title, release_date, timestamp),])
db.insert_data('reviews_raw', reviews_list)
db.close_connection()